# Import Packages

In [1]:
from zipfile import ZipFile
import xml.sax, xml.sax.handler
import os
import numpy as np
import xarray as xr
import pickle
from shapely.geometry import Polygon, LineString
import utm
from pyproj import Proj
import matplotlib.pyplot as plt
import matplotlib.colors as colors
plt.rcParams['figure.figsize'] = 18, 9
plt.rcParams['figure.facecolor'] = 'white'

# Define functions

In [2]:
def get_flux(p,u,v,z,g,rho_f,z0,kappa,fluxconstant,usimth,pth): #martin and kok 2017

    U  = (u**2+v**2)**0.5
    us = U*kappa/np.log(z/z0)
    
    qm = np.copy(fluxconstant*usimth/g*rho_f*(us**2-usimth**2))
    qa = np.copy(np.arctan2(v,u))
    
    qm[(us<usimth)|(p>pth)] = 0
    qa[(us<usimth)|(p>pth)] = np.nan
    return qm,qa

def get_impact_threshold(d,g,rho_f,rho_s,thresholdconstant): #bagnold 1941
    return thresholdconstant*((rho_s-rho_f)/rho_f*g*d)**0.5

In [8]:
scenario = 'historical'
year = 2000

path = '/media/synology3/cmip6/ec-earth3-'+scenario+'/'
pdirs = []
udirs = []
vdirs = []
for file in os.listdir(path):
    if file.startswith('.') or file.startswith('wget'):
        continue
    else:
        if int(file.split('_')[-1][:4])==year:
            if file.startswith('uas'):
                udirs.append(file)
            elif file.startswith('vas'):
                vdirs.append(file)
            elif file.startswith('pr'):
                pdirs.append(file)
udirs = sorted(udirs)
vdirs = sorted(vdirs)
pdirs = sorted(pdirs)

In [14]:
def flux_from_files(scenario,year):
    path = '/media/synology3/cmip6/ec-earth3-'+scenario+'/'
    pdirs = []
    udirs = []
    vdirs = []
    for file in os.listdir(path):
        if file.startswith('.') or file.startswith('wget'):
            continue
        else:
            if int(file.split('-')[2][:4])==year:
                if file.startswith('uas'):
                    udirs.append(file)
                elif file.startswith('vas'):
                    vdirs.append(file)
                elif file.startswith('pr'):
                    pdirs.append(file)
    udirs = sorted(udirs)
    vdirs = sorted(vdirs)
    pdirs = sorted(pdirs)

    flag = []
    if year==1850: # because some runs start early so make sure to take the shorter ones
        temp = xr.open_dataset(path+udirs[-1])
    else:
        temp = xr.open_dataset(path+udirs[0])
        
    t = np.asarray(temp.time).astype('float')*1e-9
    q = np.empty((len(udirs),2,len(t),len(Dunefield_tileinds)))
    for k in range(len(udirs)):
        try:
            p = np.asarray(xr.open_dataset(path+pdirs[k]).pr)
        except:
            p = np.ones(np.shape(xr.open_dataset(path+pdirs[k]).pr))*np.nan
            flag.append('p %d'%k)
        try:
            u = np.asarray(xr.open_dataset(path+udirs[k]).uas)
        except:
            u = np.ones(np.shape(xr.open_dataset(path+udirs[k]).uas))*np.nan
            flag.append('u %d'%k)
        try:    
            v = np.asarray(xr.open_dataset(path+vdirs[k]).vas)
        except:
            v = np.ones(np.shape(xr.open_dataset(path+vdirs[k]).vas))*np.nan
            flag.append('v %d'%k)        

        if len(p)<len(t):
            temp = np.repeat(p[np.newaxis,0],np.abs(len(t)-len(p)),axis=0)
            p = np.concatenate((temp,p))
            flag.append('p<t %d'%k)
        if len(u)<len(t):
            temp = np.repeat(u[np.newaxis,0],np.abs(len(t)-len(u)),axis=0)
            u = np.concatenate((temp,u))
            flag.append('u<t %d'%k)
        if len(v)<len(t):
            temp = np.repeat(v[np.newaxis,0],np.abs(len(t)-len(v)),axis=0)
            v = np.concatenate((temp,v))
            flag.append('v<t %d'%k)

        if len(p)>len(t):
            if year==1850:
                p = p[np.abs(len(t)-len(p)):]
            else:
                p = p[:-np.abs(len(t)-len(p))]
            flag.append('p>t %d'%k)
        if len(u)>len(t):
            if year==1850:
                u = u[np.abs(len(t)-len(u)):]
            else:
                u = u[:-np.abs(len(t)-len(u))]
            flag.append('u>t %d'%k)
        if len(v)>len(t):
            if year==1850:
                v = v[np.abs(len(t)-len(v)):]
            else:
                v = v[:-np.abs(len(t)-len(v))]
            flag.append('v>t %d'%k)

        q[k] = get_flux(p[:,Dunefield_tileinds[:,1],Dunefield_tileinds[:,0]],
                    u[:,Dunefield_tileinds[:,1],Dunefield_tileinds[:,0]],
                    v[:,Dunefield_tileinds[:,1],Dunefield_tileinds[:,0]],z,g,rho_f,z0,kappa,fluxconstant,usimth,pth)

        print('%d of %d runs loaded'%((k+1),len(udirs)),end='\r')
    
    return q,t,flag

# Define constants

In [15]:
d     = 300e-6
z     = 10
g     = 9.8
z0    = 1e-3
rho_s = 2650
rho_f = 1.2
kappa = 0.4

fluxconstant      = 5
thresholdconstant = 0.082

usimth = get_impact_threshold(d,g,rho_f,rho_s,thresholdconstant)
pth    = 1e-4

In [16]:
with open('ec-earth3-firstdim-info.pkl', 'rb') as f:
    _,Dunefield_tileinds,_,_ = pickle.load(f)

# Initialize & process data

In [17]:
scenarios = ['historical','ssp126','ssp245','ssp370','ssp585']
years = [range(1850,2014+1),range(2015,2100+1),range(2015,2100+1),range(2015,2100+1),range(2015,2100+1)]

In [7]:
for i in range(len(scenarios)):
    for j in range(len(years[i])):
        q,t,flag = flux_from_files(scenarios[i],years[i][j])
        with open('/media/synology3/cmip6/ec-earth3-flux-processed/'+scenarios[i]+'-'+str(years[i][j])+'.pkl', 'wb') as f:
            pickle.dump((q,t,flag),f)
        print('%d of %d years loaded'%((j+1),len(years[i])),end='\r')